In [37]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

df = pd.read_json('../data/final_ingredient_data.json')

In [38]:
df

,title,url,allTimeScore,view,totalLike,photos,ingredients,tags,is_has_amoungs
0,ผัดวุ้นเส้นกระเพราหมูกรอบ,https://www.wongnai.com/recipes/ugc/273249a788...,1939,1939,0,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'วุ้นเส้น', 'amount': 1, '...","[{'externalId': 'pork-recipes', 'primaryName':...",1.000000
1,ไข่ตุ๋นมาม่า,https://www.wongnai.com/recipes/ugc/d2ca1bdc51...,1519,1179,5,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ไข่ไก่', 'amount': 2, 'un...","[{'externalId': 'egg-recipes', 'primaryName': ...",1.000000
2,เกี่ยมอี๋คั่วไส้หมูก้อน,https://www.wongnai.com/recipes/ugc/ffbd2984f0...,946,826,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เกี่ยมอี๋ลวก', 'amount': ...","[{'externalId': 'main-dish-recipes', 'primaryN...",1.000000
3,ออรินจิคั่วเกลือพริกไทย,https://www.wongnai.com/recipes/ugc/66e7abffe9...,1862,1792,4,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เห็ด', 'amount': 3, 'unit...","[{'externalId': 'chicken-recipes', 'primaryNam...",1.000000
4,กระหล่ำปลีทอดน้ำปลา,https://www.wongnai.com/recipes/ugc/f0e49ced08...,1365,1305,3,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กะหล่ำปลี', 'amount': 10,...","[{'externalId': 'vegetable-recipes', 'primaryN...",1.000000
...,...,...,...,...,...,...,...,...,...
2782,สุกี้น้ำกระดูกอ่อน,https://www.wongnai.com/recipes/ugc/7087159c34...,628,398,8,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กระดูกอ่อน', 'amount': 0....","[{'externalId': 'vegetable-recipes', 'primaryN...",0.545455
2783,ผัดผักรวมมิตร,https://www.wongnai.com/recipes/ugc/5887aa7205...,987,947,1,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ดอกกะหล่ำ', 'amount': 1, ...","[{'externalId': 'vegetable-recipes', 'primaryN...",0.538462
2784,ผัดมาม่าต้มยำ,https://www.wongnai.com/recipes/ugc/b30648906c...,397,377,2,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กุ้ง', 'amount': 6, 'unit...","[{'externalId': 'noodle-recipes', 'primaryName...",0.538462
2785,สปาเก็ตตี้ขี้เมาเห็ด,https://www.wongnai.com/recipes/ugc/95804aa0df...,877,817,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เส้นสปาเก็ตตี้', 'amount'...","[{'externalId': 'main-dish-recipes', 'primaryN...",0.533333


In [39]:
uniq_external_id = df['tags'].apply(lambda tags: [tag["externalId"] for tag in tags if tag["externalId"] != 'main-dish-recipes']).tolist()

In [40]:
uniq_external_id = list(set(sum(uniq_external_id, [])))

In [41]:
uniq_external_id

['bacon-recipes',
 'salted-egg-recipes',
 'high-fiber-recipes',
 '1-3-year-baby-recipes',
 'fusion-food-recipes',
 'thai-food-recipes',
 '3-7-year-kid-recipes',
 'clean-food-recipes',
 'steak-recipes',
 'thai-noodle-recipes',
 'sinocookingclub',
 'electrolux',
 'thai-spicy-salad-recipes',
 'bread-recipes',
 'side-dish-recipes',
 'low-fat-recipes',
 'breakfast-recipes',
 'chinese-food-recipes',
 'duck-recipes',
 'chili-paste-recipes',
 'rice-cooker-recipes',
 'chicken-recipes',
 'fish-recipes',
 'egg-recipes',
 'Valentines-recipe',
 'baked-recipes',
 'steamed-recipes',
 'convection-oven-recipes',
 'mushroom-recipes',
 'noodle-recipes',
 'International-Recipe',
 'korean-food-recipes',
 'weight-loss-recipes',
 'salmon-recipes',
 'share-recipe-activity-by-knorr',
 'mothers-day-recipe',
 'low-calorie-recipes',
 'southern-thai-food-recipes',
 'sauce-recipes',
 'belly-pork-recipes',
 'smoothie-recipes',
 'tofu-menu-recipes',
 'Shrimp-Recipes',
 'recipes-for-kids-over-7-yrs',
 'cc-pork',
 'cc-

In [42]:
df['tags2'] = df['tags'].apply(lambda tags: [tag['externalId'] for tag in tags])

In [43]:
df[['tags2']]

,tags2
0,"[pork-recipes, main-dish-recipes, fried-recipe..."
1,"[egg-recipes, Mothers-Day-Activity-sponsored-b..."
2,"[main-dish-recipes, fried-recipes]"
3,"[chicken-recipes, vegetable-recipes, main-dish..."
4,"[vegetable-recipes, main-dish-recipes, fried-r..."
...,...
2782,"[vegetable-recipes, main-dish-recipes, boiled-..."
2783,"[vegetable-recipes, breakfast-recipes, main-di..."
2784,"[noodle-recipes, main-dish-recipes, fried-reci..."
2785,"[main-dish-recipes, vegetarian-recipes, Recipe..."


In [45]:
def is_external_id_in_list(external_ids, check_external_id):
  if(check_external_id in external_ids):
    return 1
  else:
    return 0

for external_id in uniq_external_id:
  df[external_id] = df['tags2'].apply(lambda x : is_external_id_in_list(x,external_id))

In [46]:
df

,title,url,allTimeScore,view,totalLike,photos,ingredients,tags,is_has_amoungs,tags2,...,budget-recipes,seafood-recipes,chicken-breast-recipes,6-12-month-baby-recipes,microwave-recipes,money-making-recipes,sausage-recipes,salad-recipes,shell-recipes,fried-recipes
0,ผัดวุ้นเส้นกระเพราหมูกรอบ,https://www.wongnai.com/recipes/ugc/273249a788...,1939,1939,0,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'วุ้นเส้น', 'amount': 1, '...","[{'externalId': 'pork-recipes', 'primaryName':...",1.000000,"[pork-recipes, main-dish-recipes, fried-recipe...",...,0,0,0,0,0,0,0,0,0,1
1,ไข่ตุ๋นมาม่า,https://www.wongnai.com/recipes/ugc/d2ca1bdc51...,1519,1179,5,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ไข่ไก่', 'amount': 2, 'un...","[{'externalId': 'egg-recipes', 'primaryName': ...",1.000000,"[egg-recipes, Mothers-Day-Activity-sponsored-b...",...,0,0,0,0,1,0,0,0,0,0
2,เกี่ยมอี๋คั่วไส้หมูก้อน,https://www.wongnai.com/recipes/ugc/ffbd2984f0...,946,826,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เกี่ยมอี๋ลวก', 'amount': ...","[{'externalId': 'main-dish-recipes', 'primaryN...",1.000000,"[main-dish-recipes, fried-recipes]",...,0,0,0,0,0,0,0,0,0,1
3,ออรินจิคั่วเกลือพริกไทย,https://www.wongnai.com/recipes/ugc/66e7abffe9...,1862,1792,4,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เห็ด', 'amount': 3, 'unit...","[{'externalId': 'chicken-recipes', 'primaryNam...",1.000000,"[chicken-recipes, vegetable-recipes, main-dish...",...,0,0,0,0,0,0,0,0,0,1
4,กระหล่ำปลีทอดน้ำปลา,https://www.wongnai.com/recipes/ugc/f0e49ced08...,1365,1305,3,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กะหล่ำปลี', 'amount': 10,...","[{'externalId': 'vegetable-recipes', 'primaryN...",1.000000,"[vegetable-recipes, main-dish-recipes, fried-r...",...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,สุกี้น้ำกระดูกอ่อน,https://www.wongnai.com/recipes/ugc/7087159c34...,628,398,8,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กระดูกอ่อน', 'amount': 0....","[{'externalId': 'vegetable-recipes', 'primaryN...",0.545455,"[vegetable-recipes, main-dish-recipes, boiled-...",...,0,0,0,0,0,0,0,0,0,0
2783,ผัดผักรวมมิตร,https://www.wongnai.com/recipes/ugc/5887aa7205...,987,947,1,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ดอกกะหล่ำ', 'amount': 1, ...","[{'externalId': 'vegetable-recipes', 'primaryN...",0.538462,"[vegetable-recipes, breakfast-recipes, main-di...",...,0,0,0,0,0,0,0,0,0,0
2784,ผัดมาม่าต้มยำ,https://www.wongnai.com/recipes/ugc/b30648906c...,397,377,2,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กุ้ง', 'amount': 6, 'unit...","[{'externalId': 'noodle-recipes', 'primaryName...",0.538462,"[noodle-recipes, main-dish-recipes, fried-reci...",...,0,0,0,0,0,0,0,0,0,1
2785,สปาเก็ตตี้ขี้เมาเห็ด,https://www.wongnai.com/recipes/ugc/95804aa0df...,877,817,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เส้นสปาเก็ตตี้', 'amount'...","[{'externalId': 'main-dish-recipes', 'primaryN...",0.533333,"[main-dish-recipes, vegetarian-recipes, Recipe...",...,0,0,0,0,0,0,0,0,0,0


In [47]:
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0])

if module_path not in sys.path:
    sys.path.append(module_path)

In [48]:
from dictionary.ingredient_dict import ingredient_dict
from dictionary.unit_dict import measure, amount_for_uncount_unit

In [49]:
def set_amount(ingredient):
  if ingredient['unit'] in measure:
    return ingredient['amount']*measure[ingredient['unit']] 
  elif ingredient['unit'] in amount_for_uncount_unit[ingredient['ingredientName']]:
    return ingredient['amount']*amount_for_uncount_unit[ingredient['ingredientName']][ingredient['unit']]
  return ingredient['amount']

In [50]:
def is_ingredients_in_list(ingredients, ingre_name):
  for ingredient in ingredients:
    if ingredient['ingredientName'] == ingre_name:
      return set_amount(ingredient)
  return 0

In [15]:
# is_ingredients_in_list([{'ingredientName': 'วุ้นเส้น', 'amount': 1, 'unit': 'ซอง'},
#  {'ingredientName': 'หมูกรอบ', 'amount': 1, 'unit': 'cup'},
#  {'ingredientName': 'ใบกะเพรา', 'amount': 1, 'unit': 'cup'},
#  {'ingredientName': 'พริก', 'amount': 4, 'unit': 'เม็ด'},
#  {'ingredientName': 'กระเทียม', 'amount': 4, 'unit': 'lobe'},
#  {'ingredientName': 'ซีอิ๊วขาว', 'amount': 0.5, 'unit': 'table_spoon'},
#  {'ingredientName': 'น้ำตาล', 'amount': 1, 'unit': 'table_spoon'},
#  {'ingredientName': 'ผงปรุงรส', 'amount': 0.5, 'unit': 'table_spoon'},
#  {'ingredientName': 'น้ำเปล่า', 'amount': 2.125, 'unit': 'table_spoon'},
#  {'ingredientName': 'น้ำมัน', 'amount': 1, 'unit': 'table_spoon'}],'พริก')

In [52]:
for ingredient_name in ingredient_dict.values():
  df[ingredient_name] = df['ingredients'].apply(lambda x : is_ingredients_in_list(x,ingredient_name))

In [53]:
df

,title,url,allTimeScore,view,totalLike,photos,ingredients,tags,is_has_amoungs,tags2,...,เพนเน่,ดอกโสน,ซูกินี่,ยีสต์,ผักขม,ถั่วแขก,ลูกกระวาน,น้ำอาจาด,ข้าวมัน,มันบด
0,ผัดวุ้นเส้นกระเพราหมูกรอบ,https://www.wongnai.com/recipes/ugc/273249a788...,1939,1939,0,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'วุ้นเส้น', 'amount': 1, '...","[{'externalId': 'pork-recipes', 'primaryName':...",1.000000,"[pork-recipes, main-dish-recipes, fried-recipe...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
1,ไข่ตุ๋นมาม่า,https://www.wongnai.com/recipes/ugc/d2ca1bdc51...,1519,1179,5,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ไข่ไก่', 'amount': 2, 'un...","[{'externalId': 'egg-recipes', 'primaryName': ...",1.000000,"[egg-recipes, Mothers-Day-Activity-sponsored-b...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2,เกี่ยมอี๋คั่วไส้หมูก้อน,https://www.wongnai.com/recipes/ugc/ffbd2984f0...,946,826,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เกี่ยมอี๋ลวก', 'amount': ...","[{'externalId': 'main-dish-recipes', 'primaryN...",1.000000,"[main-dish-recipes, fried-recipes]",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
3,ออรินจิคั่วเกลือพริกไทย,https://www.wongnai.com/recipes/ugc/66e7abffe9...,1862,1792,4,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เห็ด', 'amount': 3, 'unit...","[{'externalId': 'chicken-recipes', 'primaryNam...",1.000000,"[chicken-recipes, vegetable-recipes, main-dish...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
4,กระหล่ำปลีทอดน้ำปลา,https://www.wongnai.com/recipes/ugc/f0e49ced08...,1365,1305,3,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กะหล่ำปลี', 'amount': 10,...","[{'externalId': 'vegetable-recipes', 'primaryN...",1.000000,"[vegetable-recipes, main-dish-recipes, fried-r...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,สุกี้น้ำกระดูกอ่อน,https://www.wongnai.com/recipes/ugc/7087159c34...,628,398,8,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กระดูกอ่อน', 'amount': 0....","[{'externalId': 'vegetable-recipes', 'primaryN...",0.545455,"[vegetable-recipes, main-dish-recipes, boiled-...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2783,ผัดผักรวมมิตร,https://www.wongnai.com/recipes/ugc/5887aa7205...,987,947,1,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ดอกกะหล่ำ', 'amount': 1, ...","[{'externalId': 'vegetable-recipes', 'primaryN...",0.538462,"[vegetable-recipes, breakfast-recipes, main-di...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2784,ผัดมาม่าต้มยำ,https://www.wongnai.com/recipes/ugc/b30648906c...,397,377,2,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กุ้ง', 'amount': 6, 'unit...","[{'externalId': 'noodle-recipes', 'primaryName...",0.538462,"[noodle-recipes, main-dish-recipes, fried-reci...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2785,สปาเก็ตตี้ขี้เมาเห็ด,https://www.wongnai.com/recipes/ugc/95804aa0df...,877,817,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เส้นสปาเก็ตตี้', 'amount'...","[{'externalId': 'main-dish-recipes', 'primaryN...",0.533333,"[main-dish-recipes, vegetarian-recipes, Recipe...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0


In [55]:
df['น้ำปลา']

0        0.0
1       10.0
2       10.0
3        0.0
4       20.0
        ... 
2782     0.0
2783     0.0
2784     0.0
2785     0.0
2786     0.0
Name: น้ำปลา, Length: 2787, dtype: float64

In [56]:
import numpy as np

In [57]:
min_by_tag = {}
mean_by_tag = {}

In [58]:
def mean_and_min_by_tag(d_frame,tag):
  d_frame = d_frame[d_frame[tag] == 1]
  min_by_tag[tag] = {}
  mean_by_tag[tag] = {}
  for ingre_name in ingredient_dict.values():
    df_ingre = d_frame[d_frame[ingre_name] > 0.1]
    q_low = df_ingre[ingre_name].quantile(0.01)
    q_hi  = df_ingre[ingre_name].quantile(0.99)
    df_ingre_no_out_line = df_ingre[(df_ingre[ingre_name] < q_hi) & (df_ingre[ingre_name] > q_low)]
    mean = df_ingre_no_out_line[ingre_name].mean()
    minn = df_ingre_no_out_line[ingre_name].min()
    if not np.isnan(mean):
      mean_by_tag[tag][ingre_name] = float(mean)
    if not np.isnan(minn):
      min_by_tag[tag][ingre_name] = float(minn)


In [59]:
# about 1.5 mins
for external_id in uniq_external_id:
  mean_and_min_by_tag(df,external_id)

In [63]:
mean_by_tag

{'bacon-recipes': {'น้ำปลา': 30.0,
  'กระเทียม': 34.63636363636363,
  'ไข่ไก่': 100.0,
  'น้ำตาล': 4.642857142857143,
  'เกลือ': 5.714285714285714,
  'ซอสปรุงรส': 20.0,
  'น้ำมัน': 22.5,
  'ซีอิ๊วขาว': 11.666666666666666,
  'เนื้อหมู': 100.0,
  'พริก': 12.767857142857142,
  'ใบโหระพา': 41.666666663,
  'เส้นสปาเก็ตตี้': 182.0,
  'เบคอน': 109.94117647058823,
  'ชีส': 20.0,
  'มักกะโรนี': 300.0},
 'salted-egg-recipes': {'น้ำปลา': 21.666666666666668,
  'กระเทียม': 19.8,
  'ไข่ไก่': 162.5,
  'น้ำตาล': 10.0,
  'ซอสปรุงรส': 15.0,
  'น้ำมัน': 30.0,
  'พริก': 14.0},
 'high-fiber-recipes': {'น้ำปลา': 13.333333333333334,
  'กระเทียม': 25.125,
  'ไข่ไก่': 114.28571428571429,
  'น้ำตาล': 8.333333333333334,
  'เกลือ': 7.5,
  'ซอสปรุงรส': 23.333333333333332,
  'น้ำมันหอย': 25.0,
  'น้ำมัน': 50.45454545454545,
  'น้ำเปล่า': 229.16666666575,
  'ซีอิ๊วขาว': 40.0,
  'ต้นหอม': 26.0,
  'แครอท': 37.75,
  'เนื้อหมู': 200.0,
  'น้ำมะนาว': 37.5,
  'มะเขือเทศ': 57.0,
  'หอมใหญ่': 75.0,
  'ผักชี': 10.0,
  'พริก'

In [64]:
min_by_tag

{'bacon-recipes': {'น้ำปลา': 30.0,
  'กระเทียม': 18.0,
  'ไข่ไก่': 100.0,
  'น้ำตาล': 2.5,
  'เกลือ': 2.5,
  'ซอสปรุงรส': 5.0,
  'น้ำมัน': 10.0,
  'ซีอิ๊วขาว': 5.0,
  'เนื้อหมู': 100.0,
  'พริก': 1.25,
  'ใบโหระพา': 10.0,
  'เส้นสปาเก็ตตี้': 60.0,
  'เบคอน': 40.0,
  'ชีส': 20.0,
  'มักกะโรนี': 300.0},
 'salted-egg-recipes': {'น้ำปลา': 10.0,
  'กระเทียม': 15.0,
  'ไข่ไก่': 50.0,
  'น้ำตาล': 10.0,
  'ซอสปรุงรส': 15.0,
  'น้ำมัน': 30.0,
  'พริก': 7.0},
 'high-fiber-recipes': {'น้ำปลา': 10.0,
  'กระเทียม': 15.0,
  'ไข่ไก่': 100.0,
  'น้ำตาล': 2.5,
  'เกลือ': 5.0,
  'ซอสปรุงรส': 15.0,
  'น้ำมันหอย': 20.0,
  'น้ำมัน': 15.0,
  'น้ำเปล่า': 50.0,
  'ซีอิ๊วขาว': 30.0,
  'ต้นหอม': 12.0,
  'แครอท': 20.0,
  'เนื้อหมู': 100.0,
  'น้ำมะนาว': 27.5,
  'มะเขือเทศ': 57.0,
  'หอมใหญ่': 50.0,
  'ผักชี': 10.0,
  'พริก': 5.0,
  'เนื้อไก่': 80.0,
  'เห็ด': 25.0,
  'ผักกาด': 50.0,
  'กะหล่ำปลี': 200.0,
  'ผักบุ้ง': 100.0,
  'ผงปรุงรส': 15.0},
 '1-3-year-baby-recipes': {'กระเทียม': 15.0,
  'ไข่ไก่': 100.0,
  'น

In [24]:
# import json

# with open('../dictionary/mean_by_tag.json','w',encoding='utf8')as file:
#   json.dump(mean_by_tag,file,ensure_ascii=False)
# with open('../dictionary/min_by_tag.json','w',encoding='utf8')as file:
#   json.dump(min_by_tag,file,ensure_ascii=False)

In [65]:
df

,title,url,allTimeScore,view,totalLike,photos,ingredients,tags,is_has_amoungs,tags2,...,เพนเน่,ดอกโสน,ซูกินี่,ยีสต์,ผักขม,ถั่วแขก,ลูกกระวาน,น้ำอาจาด,ข้าวมัน,มันบด
0,ผัดวุ้นเส้นกระเพราหมูกรอบ,https://www.wongnai.com/recipes/ugc/273249a788...,1939,1939,0,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'วุ้นเส้น', 'amount': 1, '...","[{'externalId': 'pork-recipes', 'primaryName':...",1.000000,"[pork-recipes, main-dish-recipes, fried-recipe...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
1,ไข่ตุ๋นมาม่า,https://www.wongnai.com/recipes/ugc/d2ca1bdc51...,1519,1179,5,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ไข่ไก่', 'amount': 2, 'un...","[{'externalId': 'egg-recipes', 'primaryName': ...",1.000000,"[egg-recipes, Mothers-Day-Activity-sponsored-b...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2,เกี่ยมอี๋คั่วไส้หมูก้อน,https://www.wongnai.com/recipes/ugc/ffbd2984f0...,946,826,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เกี่ยมอี๋ลวก', 'amount': ...","[{'externalId': 'main-dish-recipes', 'primaryN...",1.000000,"[main-dish-recipes, fried-recipes]",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
3,ออรินจิคั่วเกลือพริกไทย,https://www.wongnai.com/recipes/ugc/66e7abffe9...,1862,1792,4,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เห็ด', 'amount': 3, 'unit...","[{'externalId': 'chicken-recipes', 'primaryNam...",1.000000,"[chicken-recipes, vegetable-recipes, main-dish...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
4,กระหล่ำปลีทอดน้ำปลา,https://www.wongnai.com/recipes/ugc/f0e49ced08...,1365,1305,3,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กะหล่ำปลี', 'amount': 10,...","[{'externalId': 'vegetable-recipes', 'primaryN...",1.000000,"[vegetable-recipes, main-dish-recipes, fried-r...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,สุกี้น้ำกระดูกอ่อน,https://www.wongnai.com/recipes/ugc/7087159c34...,628,398,8,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กระดูกอ่อน', 'amount': 0....","[{'externalId': 'vegetable-recipes', 'primaryN...",0.545455,"[vegetable-recipes, main-dish-recipes, boiled-...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2783,ผัดผักรวมมิตร,https://www.wongnai.com/recipes/ugc/5887aa7205...,987,947,1,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ดอกกะหล่ำ', 'amount': 1, ...","[{'externalId': 'vegetable-recipes', 'primaryN...",0.538462,"[vegetable-recipes, breakfast-recipes, main-di...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2784,ผัดมาม่าต้มยำ,https://www.wongnai.com/recipes/ugc/b30648906c...,397,377,2,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กุ้ง', 'amount': 6, 'unit...","[{'externalId': 'noodle-recipes', 'primaryName...",0.538462,"[noodle-recipes, main-dish-recipes, fried-reci...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2785,สปาเก็ตตี้ขี้เมาเห็ด,https://www.wongnai.com/recipes/ugc/95804aa0df...,877,817,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เส้นสปาเก็ตตี้', 'amount'...","[{'externalId': 'main-dish-recipes', 'primaryN...",0.533333,"[main-dish-recipes, vegetarian-recipes, Recipe...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0


In [66]:
def get_value_from_tag(ingre,tags,min = False):
  value = []
  if min:
    data = min_by_tag
  else:
    data = mean_by_tag
  for tag in tags:
    if ingre in data[tag]:
      value.append(data[tag][ingre])
  mean = np.array(value).mean()
  if np.isnan(mean):
    return 0
  else: return mean


In [67]:
# get_value_from_tag('น้ำตาล',["Vegan-Recipe",'cc-pork'],min=True)

In [68]:
df[['tags2']].sample(15)

,tags2
25,"[pork-recipes, main-dish-recipes, baby-recipes..."
1196,"[vegetable-recipes, main-dish-recipes, side-di..."
2136,"[noodle-recipes, main-dish-recipes, fusion-foo..."
205,"[chicken-recipes, main-dish-recipes, weight-lo..."
174,[main-dish-recipes]
1408,"[noodle-recipes, main-dish-recipes, fried-reci..."
2759,"[egg-recipes, pork-recipes, main-dish-recipes,..."
629,"[pork-recipes, main-dish-recipes, baby-recipes..."
1487,"[seafood-recipes, main-dish-recipes, thai-food..."
2275,"[egg-recipes, chicken-recipes, main-dish-recip..."


In [69]:
tags = df['tags2'].tolist()

In [70]:
tags = [[t for t in x if t != 'main-dish-recipes'] for x in tags]

In [71]:
tags

[['pork-recipes',
  'fried-recipes',
  'Recipe-Activity-sponsored-by-Knorr',
  'knorr-recipes'],
 ['egg-recipes',
  'Mothers-Day-Activity-sponsored-by-Knorr',
  'microwave-recipes',
  'weight-loss-recipes'],
 ['fried-recipes'],
 ['chicken-recipes',
  'vegetable-recipes',
  'fried-recipes',
  'Recipe-Activity-sponsored-by-Knorr'],
 ['vegetable-recipes',
  'fried-recipes',
  'weight-loss-recipes',
  'Mothers-Day-Activity-sponsored-by-Knorr'],
 ['thai-food-recipes', 'Shrimp-Recipes'],
 ['fish-recipes', 'side-dish-recipes', 'deep-fried-recipes'],
 ['pork-recipes', 'noodle-recipes', 'baby-recipes', 'Christmas-Recipe'],
 ['breakfast-recipes',
  'steak-recipes',
  'deep-fried-recipes',
  'belly-pork-recipes'],
 ['noodle-recipes', 'fried-recipes', 'share-recipe-activity-by-knorr'],
 ['fish-recipes',
  'japanese-food-recipes',
  'deep-fried-recipes',
  'sinocookingclub'],
 ['fish-recipes', 'thai-spicy-salad-recipes'],
 ['share-recipe-activity-by-knorr', 'boiled-recipes'],
 ['egg-recipes', 'dorm

In [72]:
for x in df:
  if x not in ingredient_dict.values():
    continue
  for y in range(len(df)):
    if df[x][y] == -1:
      df[x][y] = get_value_from_tag(x,tags[y],min=False)
      print(df[x][y])
    if df[x][y] == -2:
      df[x][y] = get_value_from_tag(x,tags[y],min=True)
      print(df[x][y])

/var/folders/y4/ns3s970s7vdc16tn3zs1l6km0000gp/T/ipykernel_78518/2376192153.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x][y] = get_value_from_tag(x,tags[y],min=False)
/var/folders/y4/ns3s970s7vdc16tn3zs1l6km0000gp/T/ipykernel_78518/2376192153.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x][y] = get_value_from_tag(x,tags[y],min=True)
/var/folders/y4/ns3s970s7vdc16tn3zs1l6km0000gp/T/ipykernel_78518/2615728792.py:10: RuntimeWarning: Mean of empty slice.
  mean = np.array(value).mean()
/usr/local/Cellar/python@3.9/3.9.16/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_

17.723440663199334
21.970536338225017
5.0
18.876361376136757
5.0
4.25
18.0252449006978
18.876361376136757
18.0252449006978
21.90432098765432
5.0
15.997052079921389
5.0
21.35375494071146
18.876361376136757
0.0
23.297833376242846
21.531342461673407
5.0
21.44829209572845
19.344048450966692
18.653426044330775
24.06212300550412
5.833333333333333
4.583333333333333
21.799295364546936
24.023857699381672
17.372747415732018
5.0
4.375
18.362771739130434
16.4128673510467
24.107379318646924
4.0625
19.863969887554795
17.23539333907551
21.852267008856685
6.875
6.666666666666667
18.362771739130434
19.74975039157965
29.492924528301884
9.166666666666666
5.3125
5.833333333333333
17.63303633822502
27.52146226415094
5.0
20.381089831716128
0.0
5.0
19.933858918711433
17.916666666666668
5.0
25.130947172070748
5.3125
4.375
17.496687912811485
5.833333333333333
15.941911567364466
22.43972746331237
26.964654816039687
6.875
31.725373027320884
34.61462407425458
32.495279991062446
0.0
9.375
3.5
4.75
44.0429185722807

In [75]:
df

,title,url,allTimeScore,view,totalLike,photos,ingredients,tags,is_has_amoungs,tags2,...,เพนเน่,ดอกโสน,ซูกินี่,ยีสต์,ผักขม,ถั่วแขก,ลูกกระวาน,น้ำอาจาด,ข้าวมัน,มันบด
0,ผัดวุ้นเส้นกระเพราหมูกรอบ,https://www.wongnai.com/recipes/ugc/273249a788...,1939,1939,0,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'วุ้นเส้น', 'amount': 1, '...","[{'externalId': 'pork-recipes', 'primaryName':...",1.000000,"[pork-recipes, main-dish-recipes, fried-recipe...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
1,ไข่ตุ๋นมาม่า,https://www.wongnai.com/recipes/ugc/d2ca1bdc51...,1519,1179,5,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ไข่ไก่', 'amount': 2, 'un...","[{'externalId': 'egg-recipes', 'primaryName': ...",1.000000,"[egg-recipes, Mothers-Day-Activity-sponsored-b...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2,เกี่ยมอี๋คั่วไส้หมูก้อน,https://www.wongnai.com/recipes/ugc/ffbd2984f0...,946,826,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เกี่ยมอี๋ลวก', 'amount': ...","[{'externalId': 'main-dish-recipes', 'primaryN...",1.000000,"[main-dish-recipes, fried-recipes]",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
3,ออรินจิคั่วเกลือพริกไทย,https://www.wongnai.com/recipes/ugc/66e7abffe9...,1862,1792,4,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เห็ด', 'amount': 3, 'unit...","[{'externalId': 'chicken-recipes', 'primaryNam...",1.000000,"[chicken-recipes, vegetable-recipes, main-dish...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
4,กระหล่ำปลีทอดน้ำปลา,https://www.wongnai.com/recipes/ugc/f0e49ced08...,1365,1305,3,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กะหล่ำปลี', 'amount': 10,...","[{'externalId': 'vegetable-recipes', 'primaryN...",1.000000,"[vegetable-recipes, main-dish-recipes, fried-r...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,สุกี้น้ำกระดูกอ่อน,https://www.wongnai.com/recipes/ugc/7087159c34...,628,398,8,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กระดูกอ่อน', 'amount': 0....","[{'externalId': 'vegetable-recipes', 'primaryN...",0.545455,"[vegetable-recipes, main-dish-recipes, boiled-...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2783,ผัดผักรวมมิตร,https://www.wongnai.com/recipes/ugc/5887aa7205...,987,947,1,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'ดอกกะหล่ำ', 'amount': 1, ...","[{'externalId': 'vegetable-recipes', 'primaryN...",0.538462,"[vegetable-recipes, breakfast-recipes, main-di...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2784,ผัดมาม่าต้มยำ,https://www.wongnai.com/recipes/ugc/b30648906c...,397,377,2,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'กุ้ง', 'amount': 6, 'unit...","[{'externalId': 'noodle-recipes', 'primaryName...",0.538462,"[noodle-recipes, main-dish-recipes, fried-reci...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2785,สปาเก็ตตี้ขี้เมาเห็ด,https://www.wongnai.com/recipes/ugc/95804aa0df...,877,817,6,[{'contentUrl': 'https://img.wongnai.com/p/196...,"[{'ingredientName': 'เส้นสปาเก็ตตี้', 'amount'...","[{'externalId': 'main-dish-recipes', 'primaryN...",0.533333,"[main-dish-recipes, vegetarian-recipes, Recipe...",...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0


In [77]:
df_model = df.drop(['url','allTimeScore','view','ingredients','photos','tags','tags2','is_has_amoungs'],axis=1)

In [81]:
df_model

,title,totalLike,bacon-recipes,salted-egg-recipes,high-fiber-recipes,1-3-year-baby-recipes,fusion-food-recipes,thai-food-recipes,3-7-year-kid-recipes,clean-food-recipes,...,เพนเน่,ดอกโสน,ซูกินี่,ยีสต์,ผักขม,ถั่วแขก,ลูกกระวาน,น้ำอาจาด,ข้าวมัน,มันบด
0,ผัดวุ้นเส้นกระเพราหมูกรอบ,0,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
1,ไข่ตุ๋นมาม่า,5,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2,เกี่ยมอี๋คั่วไส้หมูก้อน,6,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
3,ออรินจิคั่วเกลือพริกไทย,4,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
4,กระหล่ำปลีทอดน้ำปลา,3,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,สุกี้น้ำกระดูกอ่อน,8,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2783,ผัดผักรวมมิตร,1,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2784,ผัดมาม่าต้มยำ,2,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0
2785,สปาเก็ตตี้ขี้เมาเห็ด,6,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0.0


In [84]:
df_model.to_csv('../data/data_for_model.csv',index=False)

In [89]:
category_tag = {}

In [90]:
for x in df['tags']:
  for tag in x:
    if tag['category'] not in category_tag:
      category_tag[tag['category']] = set()
    category_tag[tag['category']].add(tag['externalId'])


In [94]:
cate_dict = {}
for k in category_tag:
  cate_dict[k] = list(category_tag[k])

In [95]:
cate_dict

{'วัตถุดิบ': ['belly-pork-recipes',
  'bacon-recipes',
  'duck-recipes',
  'tofu-menu-recipes',
  'Shrimp-Recipes',
  'salted-egg-recipes',
  'rice-recipes',
  'fruit-recipes',
  'pork-recipes',
  'avocado-recipes',
  'cheese-recipes',
  'squid-recipes',
  'chicken-recipes',
  'fish-recipes',
  'egg-recipes',
  'mushroom-recipes',
  'noodle-recipes',
  'beef-recipes',
  'seafood-recipes',
  'vegetable-recipes',
  'crab-recipes',
  'bread-recipes',
  'chicken-breast-recipes',
  'salmon-recipes',
  'knorr-recipes',
  'sausage-recipes',
  'shell-recipes'],
 'ประเภทอาหาร': ['chili-paste-recipes',
  'isan-food-recipes',
  'vegetarian-recipes',
  'japanese-food-recipes',
  'dinner-recipes',
  'italian-food-recipes',
  'fusion-food-recipes',
  'thai-food-recipes',
  'sandwich-recipes',
  'curry-recipes',
  'thai-side-dish-recipes',
  'thai-noodle-recipes',
  'steak-recipes',
  'main-dish-recipes',
  'sauce-recipes',
  'International-Recipe',
  'korean-food-recipes',
  'western-food-recipes',


In [96]:
import json

with open('../dictionary/tag_category.json','w',encoding='utf8')as file:
  json.dump(cate_dict,file,ensure_ascii=False)